In [214]:
from SpotifyPlayer import SpotifyPlayer
player = SpotifyPlayer()
player.authenticate()

Successfully authenticated


In [26]:
import joblib

kanyeId = "5K4W6rqBFWDnAN6FQUkS6x"
kanyeDiscography = player.getArtistDiscography(kanyeId)
joblib.dump(kanyeDiscography, 'joblib/kanyeDiscography.joblib')

allTracks = set()
for albumTracks in kanyeDiscography.values():
    allTracks |= albumTracks
print("Got discography")

Got discography


In [45]:
import time
runningPlaylistResults = {}
numIterations = 500
delay = 1 # seconds
probableKanyePlaylistQueries = ["rap", "hip hop", "pop"]

for i in range(numIterations):
    for genre in probableKanyePlaylistQueries:
        runningPlaylistResults = player.playlistQuery(genre, allTracks, runningPlaylistResults)
    time.sleep(delay)
    if i % 5 == 0:
        print("Done with", i)
        print(len(runningPlaylistResults))
        
    # don't wanna our token to expire
    if i % 10 == 0:
        oldLastIndex = player.lastIndex
        player = SpotifyPlayer()
        player.authenticate()
        player.lastIndex = oldLastIndex

joblib.dump(runningPlaylistResults, 'joblib/playlistResults.joblib')
runningPlaylistResults = runningPlaylistResults.values()

print("Done!")

# import glob
# import json
# playlistJSON = glob.glob('./playlists/*.json')

# def extractTrackURI(trackString):
#     i = trackString.rfind(":")
#     return trackString[(i+1):]


# runningPlaylistResults = []
# i = 0
# j = 0
# for playlistChunk in playlistJSON:
#     j += 1
#     with open(playlistChunk) as f:
#         playlists = json.load(f)["playlists"]
#         for playlist in playlists:
#             # {"tracks", "total"}
#             results = {}
#             results["total"] = playlist["num_tracks"]
#             results["tracks"] = []

#             for track in playlist["tracks"]:
#                 uri = extractTrackURI(track["track_uri"]) 
#                 if uri in allTracks:
#                     results["tracks"].append(uri)

#             if len(results["tracks"]) > 1:
#                 runningPlaylistResults.append(results)
#                 i += 1
    
# joblib.dump(runningPlaylistResults, 'joblib/millionPlaylist.joblib')
# print(i)
# print(j)

Done with 0
Successfully authenticated
retrying ...1secs
Done with 5
Done with 10
Successfully authenticated
retrying ...1secs
Done with 15
Done with 20
Successfully authenticated
retrying ...1secs
Done with 25
Done with 30
Successfully authenticated
Done with 35
Done with 40
Successfully authenticated
Done with 45
Done with 50
Successfully authenticated
Done with 55
Done with 60
Successfully authenticated
Done with 65
retrying ...1secs
Done with 70
Successfully authenticated
Done with 75
Done with 80
Successfully authenticated
Done with 85
Done with 90
Successfully authenticated
Done with 95
Done with 100
Successfully authenticated
Done with 105
Done with 110
Successfully authenticated
Done with 115
Done with 120
Successfully authenticated
Done with 125
Done with 130
Successfully authenticated
Done with 135
retrying ...1secs
Done with 140
Successfully authenticated
Done with 145


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/playlists/2jeAd6wYHX6va0DVJ5mfyw/tracks?fields=items.track.id:
 The access token expired

In [49]:
print(len(runningPlaylistResults))
joblib.dump(runningPlaylistResults, 'joblib/playlistResults.joblib')
runningPlaylistResults = runningPlaylistResults.values()

1263


In [116]:
redundantAlbumMapping = {
    # college dropout
    ("0bSrRp502dZKkpwZa9F1MY", "4Uv86qWpGTxf7fU7lG5X6F", "3lQePoIm6iNQIiZkCYxCy0"): "3ff2p3LnR6V7m6BinwhNaQ",
    # late registration
    ("4yJqrqT2BpuXLj5BMJlAXR",): "5ll74bqtkcXlKE7wwkMq4g",
    # I love it
    ("1mTS3TI1DYu3U25RAVwwBv", "7Jq7aQPj4z2Rrh51nbBu4U",): "185Tm1g5U7eMOrm3m9SQUh",
    # MBDTF
    ("6klUp8sQyRXGuJhqZu4PG3",): "20r762YmB5HeofjMCiPMLv",
    # graduation
    ("7nxBMFwJtTTKDNxVCKmYi3",): "3SZr5Pco2oqKFORCP3WNj9"
}

redundantSongMapping = {}

for redundantAlbums in redundantAlbumMapping:
    realAlbumId = redundantAlbumMapping[redundantAlbums]
    realAlbumTracks = player.sp._get("albums/{}".format(realAlbumId))["tracks"]["items"]
    realAlbumTrackIds = [track["id"] for track in realAlbumTracks]
    
    for redundantAlbumId in redundantAlbums:
        redundantAlbum = player.sp._get("albums/{}".format(redundantAlbumId))
        redundantAlbumTracks = redundantAlbum["tracks"]["items"]
        for i in range(len(redundantAlbumTracks)):
            trackId = redundantAlbumTracks[i]["id"]
            redundantSongMapping[trackId] = realAlbumTrackIds[i]

In [126]:
badAlbums = ["0A3g19AGFd9Qe3rAIkP8e0"]
badSongs = set()
for badAlbumId in badAlbums:
    badAlbum = player.sp._get("albums/{}".format(badAlbumId))
    badAlbumTracks = badAlbum["tracks"]["items"]
    for badTrack in badAlbumTracks:
        badSongs.add(badTrack["id"])

In [127]:
playlistResultList = list(runningPlaylistResults)
for i in range(len(playlistResultList)):
    currentTracks = playlistResultList[i]['tracks']
    filteredTracks = set()
    for track in currentTracks:
        if track in badSongs:
            continue
        if track in redundantSongMapping:
            filteredTracks.add(redundantSongMapping[track])
        else:
            filteredTracks.add(track)
            
    playlistResultList[i]['total'] += len(filteredTracks) - len(playlistResultList[i]["tracks"])
    playlistResultList[i]['tracks'] = list(filteredTracks)
    
filteredTracks = set()
for track in allTracks:
    if track in redundantSongMapping:
        filteredTracks.add(redundantSongMapping[track])
    else:
        filteredTracks.add(track)
allTracks = filteredTracks

In [220]:
albumArtMap = {}
albumsToIgnore = set()
for redundantAlbums in redundantAlbumMapping:
    albumsToIgnore |= set(redundantAlbums)
albumsToIgnore |= set(badAlbums)

for albumId in kanyeDiscography:
    if albumId in albumsToIgnore:
        continue
        
    albumResponse = player.sp._get("albums/{}".format(albumId))
    albumArtURL = albumResponse["release_date"] + ";" + albumResponse["images"][0]["url"]
    for songId in kanyeDiscography[albumId]:
        albumArtMap[songId] = albumArtURL


In [221]:
set(albumArtMap.values())

{'2004-02-13;https://i.scdn.co/image/aa5b579c2da0dba06b5597127644d3d46f254cdb',
 '2005-09-30;https://i.scdn.co/image/92db1dda6ed503723c89c3cf03369c908b1393a3',
 '2007-01-01;https://i.scdn.co/image/01bb4663dcfa3b8b4fd405ef493dfe447805aa57',
 '2008-01-01;https://i.scdn.co/image/353e99e5ff167272c245412b52d84bc36185b58d',
 '2010-01-01;https://i.scdn.co/image/cafd8763a050dabc4f6157172bb09a23d09abde3',
 '2013-01-01;https://i.scdn.co/image/87982737e43f86e172dbf9897f6e437fde2d7caf',
 '2016-06-10;https://i.scdn.co/image/443372cd2c6d4245833fb46ac1c5dabca00c78a9',
 '2018-06-01;https://i.scdn.co/image/05cf2f8b56e595bcbf50fccb894f5fb6c2427750',
 '2018-06-08;https://i.scdn.co/image/64cc5671890ba19c6c42a533eed08da56d29bdca',
 '2018-08-30;https://i.scdn.co/image/1bb1302e5cbce5cb96a6ca6c838c3f9c6f54456f',
 '2018-09-07;https://i.scdn.co/image/67a6a4e49285e1983e771811fcb463b4422d0e5d'}

In [128]:
import numpy as np

remainingTracks = set()

# initially track # occurrences, then use it to store the idf value
trackCounter = {}

for trackSet in runningPlaylistResults:
    remainingTracks |= set(trackSet['tracks'])
    for trackId in trackSet['tracks']:
        if trackId not in trackCounter:
            trackCounter[trackId] = 0
        trackCounter[trackId] += 1

numRemainingTracks = len(remainingTracks)
numPlaylists = len(runningPlaylistResults)
print("Found results for {}/{} songs".format(numRemainingTracks, len(allTracks)))

trackIndexer = {}
i = 0
# faster than an array for indexing
for trackId in remainingTracks:
    trackIndexer[trackId] = i
    i += 1
    trackCounter[trackId] = np.log(numPlaylists / trackCounter[trackId])

joblib.dump(trackIndexer, 'joblib/trackIndexer.joblib')

    
matrix = None
for playlistIdOutput in runningPlaylistResults:
    songArray = [0] * numRemainingTracks

    numSongsInPlaylist = playlistIdOutput['total']
    tf = 1.0 / numSongsInPlaylist

    for trackId in playlistIdOutput['tracks']:
        index = trackIndexer[trackId]
        idf = trackCounter[trackId]

        songArray[index] = tf * idf
    vector = np.array([songArray]).T
    if matrix is None:
        matrix = vector
    else:
        matrix = np.hstack((matrix, vector))

joblib.dump(matrix, 'joblib/matrix.joblib')


Found results for 126/138 songs


['joblib/matrix.joblib']

In [130]:
unpopularSongs = allTracks - remainingTracks - badSongs
print("Songs not found were")
for trackId in unpopularSongs:
    trackInfo = player.sp._get("tracks/{}".format(trackId))
    print("*", trackInfo['name'])

Songs not found were


In [131]:
indexToTrack = np.array(sorted(trackIndexer, key=trackIndexer.get))

totalValueArray = np.array([-1 * sum([el ** 2 for el in row]) for row in matrix])
k = 10

topKSongs = np.argpartition(totalValueArray, kth=k)[:k]
sortedTopKSongs = topKSongs[np.argsort(totalValueArray[topKSongs])]
print("top songs")
i = 0
for trackId in indexToTrack[sortedTopKSongs]:
    trackInfo = player.sp._get("tracks/{}".format(trackId))
    print(str(i+1) + ".", trackInfo['name'])
    i += 1

top songs
1. Runaway
2. I Love It (& Lil Pump)
3. Monster
4. All Mine
5. Street Lights
6. Blame Game
7. All Falls Down
8. Heartless
9. Reborn
10. POWER


In [132]:
targetFeatures = [
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "speechiness",
    "valence",
     "tempo"
]

featureMatrix = np.zeros((len(indexToTrack), len(targetFeatures)))

for i in range(len(indexToTrack) // 100 + 1):
    start = i * 100
    stop = (i + 1) * 100
    featureResponseChunk = player.sp._get('audio-features', ids=','.join(indexToTrack[start:stop]))


    featureArray = featureResponseChunk['audio_features']

    for songIndex in range(len(featureArray)):
        songFeatures = featureArray[songIndex]
        currentVector = []
        for targetFeature in targetFeatures:
            currentVector.append(songFeatures[targetFeature])
        featureMatrix[songIndex + start] = np.array(currentVector)

featureMatrix = np.hstack((matrix, featureMatrix))

joblib.dump(featureMatrix, 'joblib/featureMatrix.joblib')


['joblib/featureMatrix.joblib']

In [133]:
oldLastIndex
joblib.dump(oldLastIndex, 'joblib/oldLastIndexNew.joblib')

['joblib/oldLastIndex.joblib']

In [134]:
trackIdToName = {}
for trackId in indexToTrack:
    trackInfo = player.sp._get("tracks/{}".format(trackId))
    trackIdToName[trackId] = trackInfo['name']

In [280]:
numAlbumsAndSingles = len(kanyeDiscography)
numAlbums = 9
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.preprocessing import normalize


classifiersWithLabels = {
    "KMeans": KMeans
}

pcaMatrix = PCA(9).fit_transform(matrix)
featureAgglommeration = FeatureAgglomeration(9).fit_transform(matrix)
audioFeatures = featureMatrix[:,-9:]
pcaMatrixWithAudio = np.hstack((pcaMatrix, audioFeatures))
featureAgglommerationWithAudio = np.hstack((featureAgglommeration, audioFeatures))


matricesWithLabels = {
    "Raw Playlist Matrix": normalize(matrix),
    "Playlist Matrix + Audio Features": normalize(featureMatrix),
    "PCA'd Playlist Matrix": normalize(pcaMatrix),
    "PCA Playlist Matrix + Audio Features": normalize(pcaMatrixWithAudio),
    "Feature Agglomerated Playlist Matrix": normalize(pcaMatrix),
    "Feature Agglomerated Playlist Matrix + Audio Features": normalize(pcaMatrixWithAudio),
}

currentDump = 0
for classifierLabel in classifiersWithLabels:
    classifier = classifiersWithLabels[classifierLabel]
    
    for matrixLabel in matricesWithLabels:
        
        
        title = matrixLabel + " using " + classifierLabel
        albums = []
        print("Computing for",title, "\n")
        currentMatrix = matricesWithLabels[matrixLabel]
        # 300, 8008
        model = classifier(numAlbums, random_state=300)
        model.fit(currentMatrix)
        labels = model.predict(currentMatrix)
        clusteredIndices = np.argsort(labels)
        i = -1
        for index in clusteredIndices:
            currentLabel = labels[index]
            if currentLabel != i:
                i = currentLabel
                print("Album",i)
                albums.append([])
                
            trackId = indexToTrack[index]
            songTitle = trackIdToName[trackId]
            print("\t",songTitle)
            
            albums[-1].append((albumArtMap[trackId], songTitle))
        for i in range(len(albums)):
            albums[i] = sorted(albums[i])
    
        print("\n" * 3)
        with open("chart{}.json".format(currentDump), "w") as outfile:
            json.dump({"albums":albums, "title": title}, outfile)
            print("Big dump for", currentDump)
        currentDump += 1

Computing for Raw Playlist Matrix using KMeans 

Album 0
	 Coldest Winter
	 Welcome To Heartbreak
	 Bad News
	 Paranoid
	 See You In My Nightmares
	 RoboCop
	 Love Lockdown
	 Say You Will
	 Amazing
	 Pinocchio Story (Freestyle Live From Singapore)
Album 1
	 Fade
	 Big Brother
	 I Wonder
	 The Glory
	 Drunk and Hot Girls
	 Monster
	 Champion
	 Touch The Sky
	 Homecoming
	 Everything I Am
Album 2
	 Gone
	 Devil In A New Dress
	 All Falls Down
	 Celebration
	 Hey Mama
	 Addiction
	 Spaceship
	 Last Call
	 Family Business
	 Slow Jamz
	 Through The Wire
	 Heard 'Em Say
	 We Major
	 We Don't Care
	 Two Words
	 Late
	 Bound 2
	 Never Let Me Down
Album 3
	 Father Stretch My Hands Pt. 1
	 Waves
	 Saint Pablo
	 Pt. 2
	 Frank's Track
	 Famous
	 Real Friends
	 No More Parties In LA
	 Freestyle 4
	 I Love Kanye
	 Facts (Charlie Heat Version)
	 Ultralight Beam
	 Feedback
	 Highlights
	 Wolves
	 30 Hours
	 FML
Album 4
	 Fire
	 Feel The Love
	 4th Dimension
	 All Mine
	 Kids See Ghosts
	 I Love It (& 

Album 0
	 Freestyle 4
	 Workout Plan
	 I'll Fly Away
	 School Spirit
	 Siiiiiiiiilver Surffffeeeeer Intermission
	 Frank's Track
	 Intro
	 Breathe In Breathe Out
	 My Way Home
	 School Spirit Skit 2
	 Bring Me Down
	 Skit #3
	 Skit #4
	 Lil Jimmy Skit
	 School Spirit Skit 1
	 Skit #2
	 Diamonds From Sierra Leone - Bonus Track
	 Get Em High
	 Wake Up Mr. West
	 Low Lights
	 Graduation Day
	 Skit #1
	 Pinocchio Story (Freestyle Live From Singapore)
	 Crack Music
Album 1
	 Waves
	 Facts (Charlie Heat Version)
	 Pt. 2
	 Fade
	 FML
	 No More Parties In LA
	 Real Friends
	 Famous
	 I Love Kanye
	 Father Stretch My Hands Pt. 1
	 Saint Pablo
	 30 Hours
	 Ultralight Beam
	 Highlights
	 Feedback
	 Wolves
Album 2
	 We Major
	 Hey Mama
	 Family Business
	 Slow Jamz
	 Heard 'Em Say
	 Late
	 Gone
	 Last Call
	 Touch The Sky
	 All Falls Down
	 Addiction
	 Through The Wire
	 Never Let Me Down
	 Celebration
	 Devil In A New Dress
	 We Don't Care
	 Drive Slow
	 The New Workout Plan
	 Spaceship
	 Diamond

In [230]:
# from scipy.stats import pearsonr
# pcaSizes = [9, 3, 1]

# print(" | ".join(targetFeatures))
# for pcaSize in pcaSizes:
#     print("Evaluating PCA of size", pcaSize)
#     pcaMatrix = PCA(pcaSize).fit_transform(matrix)
#     print(pcaMatrix.shape)
#     for i in range(len(pcaMatrix[0])):
#         output = "|"
#         for j in range(len(audioFeatures[0])):
#             X = pcaMatrix[:,i]
#             y = audioFeatures[:,j]

#             r, p = pearsonr(X, y)
#             output += str(round(r, 2)) + "\t|"
#         print(output)

from scipy.stats import pearsonr
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

dimSizes = [3 ** i for i in range(5)]
k = 5

def evaluateCorrelation(reducedMatrix, algorithm):
    print("Evaluating dimensionality reduction of size", len(reducedMatrix[0]), "using", algorithm)
#     reducedMatrix = FeatureAgglomeration(dimSize).fit_transform(matrix)
    values = []
    
    for i in range(len(reducedMatrix[0])):
#         output = "|"
        for j in range(len(audioFeatures[0])):
            X = reducedMatrix[:,i]
            y = audioFeatures[:,j]

            r, p = pearsonr(X, y)
#             output += str(round(r, 2)) + "\t|"
            values.append((-1 * abs(r), r, i, j))
#         print(output)
    bestPredictors = sorted(values)
    for i in range(k):
        r = bestPredictors[i][1]
        j = bestPredictors[i][-1]
        print("\t",str(i+1) + ".", "r =", round(r, 2), "for", targetFeatures[j])
    print("\n")
    
algos = {
    "PCA": PCA,
    "Feature Agglomeration": FeatureAgglomeration
}

    
normalizedMatrix = normalize(matrix)
for dimSize in dimSizes:
    for algoName in algos:
        evaluateCorrelation(algos[algoName](dimSize).fit_transform(normalizedMatrix), algoName)
evaluateCorrelation(matrix, "the original matrix")


Evaluating dimensionality reduction of size 1 using PCA
	 1. r = 0.69 for speechiness
	 2. r = -0.56 for loudness
	 3. r = 0.43 for valence
	 4. r = 0.4 for acousticness
	 5. r = -0.38 for energy


Evaluating dimensionality reduction of size 1 using Feature Agglomeration
	 1. r = 0.41 for energy
	 2. r = 0.4 for loudness
	 3. r = -0.39 for acousticness
	 4. r = -0.29 for speechiness
	 5. r = -0.14 for valence


Evaluating dimensionality reduction of size 3 using PCA
	 1. r = 0.69 for speechiness
	 2. r = -0.56 for loudness
	 3. r = 0.43 for valence
	 4. r = 0.4 for acousticness
	 5. r = 0.39 for energy


Evaluating dimensionality reduction of size 3 using Feature Agglomeration
	 1. r = 0.72 for speechiness
	 2. r = -0.57 for loudness
	 3. r = 0.41 for valence
	 4. r = -0.41 for energy
	 5. r = 0.4 for loudness


Evaluating dimensionality reduction of size 9 using PCA
	 1. r = 0.69 for speechiness
	 2. r = -0.56 for loudness
	 3. r = 0.43 for valence
	 4. r = 0.4 for acousticness
	 5. r